# Platform information

In [1]:
library(benchmarkme)
get_platform_info()$OS.type
get_r_version()$version.string
get_cpu()$model_name;get_cpu()$no_of_cores
get_ram()

[1] "windows"

[1] "R version 3.5.1 (2018-07-02)"

[1] "Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz"

[1] 8

8.59 GB

# Parallel computation

In [2]:
library(doParallel)
library(foreach)
cl<-makeCluster(8) 

Loading required package: foreach
Loading required package: iterators
Loading required package: parallel


# Loading data

In [6]:
geno <-read.table("./Hetero_realigned_cov10_filtered3.raw", row.names=1, header=T)
row.names(geno)=1:240
x <- as.matrix(geno)-1   
pheno <- read.csv("./2017heteroPheno.csv", header=T)
attach(pheno)
#tbcw=sqrt(pheno$bcw+1), data normalization
data <- data.frame(tbcw=sqrt(pheno$bcw+1),length=pheno$length,gid=1:240)

The following objects are masked from pheno (pos = 3):

    bcw, gill, i5, i7, length, no, tank, vili, X



# Parameters for cross validation

In [12]:
repeats <- 10
n.fold <- 5 
n.sample <- length(pheno$bcw)    
CM<-7

# GBLUP

In [13]:
library(rrBLUP)
packageVersion("rrBLUP")

[1] '4.6'

In [14]:
#relationship matrix (Endelman at al. 2011)
A <- A.mat(x, n.core=8)
row.names(A)=1:240;colnames(A)=1:240

In [19]:
registerDoParallel(cl)
system.time({
GBLUP<-foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold,.packages="rrBLUP") %dopar% {
         bcw_test <- data
         bcw_test$tbcw[id == i] <- NA
         res <- kin.blup(bcw_test, K=A, geno="gid", pheno="tbcw")
         cor(data$tbcw[id==i],res$pred[id==i])
    }
   }
})
stopImplicitCluster()

   user  system elapsed 
   0.19    0.11    0.78 

# Bayes C

In [22]:
library("BGLR")
packageVersion("BGLR")

[1] '1.0.5'

In [28]:
registerDoParallel(cl)
system.time({
BC <- foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold,.packages="BGLR") %dopar% {
         bcw_test <- data
         bcw_test$tbcw[id == i] <- NA
         fmBC=BGLR(y=bcw_test$tbcw,ETA=list(list(X=x,model='BayesC')),nIter=2000,burnIn=1000)
         cor(data$tbcw[id == i],fmBC$yHat[id == i])
         }  
    }
})
stopImplicitCluster()  

   user  system elapsed 
   0.74    1.16   66.81 

In [39]:
#Save result
Acc<-data.frame(unlist(GBLUP),unlist(BC))
colnames(Acc)<-c("GBLUP","BC")
library("xlsx")
write.xlsx(Acc, "All_models_Acc.xlsx") 

# Comparision 

In [43]:
#Load data(from this page and deep learning models)
library("readxl")
data<-read_excel("All_models_Acc.xlsx")
Acc_all<-subset(data,select = - c(X__1))

In [44]:
(Acc_mean<-sapply(Acc_all,function(x) round(mean(x),digits = 3)))

GBLUP    BC    NN   MNN 
0.284 0.287 0.300 0.307

In [45]:
(Acc_SE<-sapply(Acc_all,function(x) round(sd(x)/sqrt(repeats*n.fold),digits = 3)))

GBLUP    BC    NN   MNN 
0.017 0.016 0.015 0.016